# <span style="color:#6e9c5a">Web Scraping con Selenium</span>

### Página:  https://statsroyale.com/

Este proyecto es un caso práctico de cómo usar el paquete Selenium para hacer webScraping a páginas web y así, recolectar información.

La página en cuestión almacena información sobre el videojuego mobile *Clash Royale*. Nosotros en específico buscamos extraer y almacenar los datos relacionados a las cartas del juego. Para ello crearemos una serie de pasos que debe seguir nuestro programa para llegar a cada página y extraer la información:
- Nombre de la carta
- Unidades
- Rareza
- Objetivos
- Rango
- Velocidad de Ataque
- Ataque


#### Primeros pasos
Primero lo primero, vamos a instalar el paquete selenium con la función `pip install selenium` en el cmd de nuestra pc. Otros paquetes que también serán necesarios son `pandas` y `re`.

In [3]:
#Importamos los paquetes que vamos a necesitar

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import selenium.webdriver.common.by
#import time
import pandas as pd
import re

### Importante: 
#### Selenium trabaja con drivers según el navegador y el sistema operativo que vayamos a utilizar, en nuestro caso usamos la versión Version: 114.0.1823.18  para [sistemas operativos basados en 64 bits](https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/). Para probar el código descarga el que se adapte a tu pc, si no estás seguro qué versión usar consulta la [documentación](https://www.selenium.dev/documentation/).

### Iniciamos el navegador
Lo que buscamos es los siguiente:
1. Ingrese a la web 'https://statsroyale.com/'

![Web de inicio](https://pbs.twimg.com/media/Fy6_OX7WwAQxQEJ?format=jpg&name=large "Logo Title Text 1")

2. Ingrese a la opción de 'cards'

![Web de cartas](https://pbs.twimg.com/media/Fy6_OX6XoAEqXzs?format=jpg&name=large "Logo Title Text 1")


3. Ingrese a la página de la primera carta Caballero ( Knight)

![Web de carta](https://pbs.twimg.com/media/Fy6_OX9WYAEQrhD?format=jpg&name=medium "Logo Title Text 1")

4. Extraiga la información solicitada y almacenar en listas
5. Regresar a la página de inicio y repetir pasos hasta cubrir con todas las cartas que queramos.
6. Crear un diccionario con las listas creadas
7. Generar una Estructura DataFrame con los datos extraidos.
8. Finalizamos creando un csv y ordenamos.

### WebScraping código

In [4]:
# Opciones de navegación

options = webdriver.EdgeOptions() #Usamos la función Options()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

#Creamos listas vacías para almacenar los datos
Name=[]
Count=[]
Rare=[]
Targ=[]
Rang=[]
Hit=[]
Sp=[]

#Asinamos las opciones configuradas a un objeto llamado driver
driver = webdriver.Edge(options = options)

# Para las primeras 6 cartas

#Creamos un bucle para las primeras 6 cartas
for i in range(1,7):

    driver.get('https://statsroyale.com/') #Abrimos la web
    
    #Paso: Darle clik a 'cartas'

    WebDriverWait(driver, 7)\
    .until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[4]/div[1]/div/div[2]/div/div[3]/a")))\
    .click()
    
    #Paso: Damos click en la carta deseada

    WebDriverWait(driver, 7)\
    .until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[4]/div[3]/div/div[4]/div[2]/div["+str(i)+"]/a")))\
    .click()
    #print("/html/body/div[4]/div[3]/div/div[4]/div[2]/div["+str(i)+"]/a")

    #Paso: Fijamos a selenium donde esta la información que queremos extraer
    
    WebDriverWait(driver, 28)\
    .until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[4]/div[3]/div/div[2]/div[2]")))

    #Almacenamos la información en un objeto 'carta'
    
    carta= driver.find_element(By.XPATH, "/html/body/div[4]/div[3]/div/div[2]/div[2]")
    carta=carta.text #Lo pasamos a str para poder manipularlo
    
    #print(carta)
    #Knight
    #A tough melee fighter. The Barbarian's handsome, cultured cousin. Rumor has it that he was knighted based on the sheer awesomeness of his mustache alone.
    #Troop , Training Camp
    #Rarity
    #Common
    #Targets
    #Ground
    #Range
    #Melee
    #Hit Speed
    #1.2sec
    #Speed
    #Medium
    
    #Guardamos el nombre de la carta
    ifcard = carta.split('\n')[0]
    A=[] #Creamos la lista vacia
    
    #Usamos regex para identificar y guardar los datos que queramos
    #En caso de no encortrar el dato guardará un 'Na

    A.append((re.search(r"Rarity\n(.+)", carta)).group(1))
    if re.search(r"Count\n(.+)", carta) != None:
        A.append((re.search(r"Count\n(.+)", carta)).group(1))
    else:
        A.append('Na')
    A.append((re.search(r"Targets\n(.+)", carta)).group(1))
    A.append((re.search(r"Range\n(.+)", carta)).group(1))
    A.append((re.search(r"Hit Speed\n(.+)", carta)).group(1))
    if re.search(r"\nSpeed\n(.+)", carta) != None:
        A.append((re.search(r"\nSpeed\n(.+)", carta)).group(1))
    else:
        A.append('Na')
        
    #Estructura de A=[Nombre, Rareza, Unidades, Objetivos, Rango, V. Ataque, Velocidad]
    #Agregamos los elementos de la lista A a las listas de las columnas a crear
    Name.append(ifcard)
    Rare.append(A[0])
    Count.append(A[1])
    Targ.append(A[2])
    Rang.append(A[3])
    Hit.append(A[4])
    Sp.append(A[5])
    A=[]


In [5]:
#Creamos un diccionario con las listas
dicti={'Nombre':Name,'Count':Count, 'Rareza':Rare, 'Objetivos':Targ, 'Rango':Rang, 'Velocidad Atk':Hit, 'Velocidad': Sp}

#Creamos un dataframe tipo pandas

df=pd.DataFrame(dicti)
df

,Nombre,Count,Rareza,Objetivos,Rango,Velocidad Atk,Velocidad
0,Knight,Na,Common,Ground,Melee,1.2sec,Medium
1,Archers,x 2,Common,Air & Ground,5.0,0.9sec,Medium
2,Goblins,x 4,Common,Ground,Melee,1.1sec,Very Fast
3,Giant,Na,Rare,Buildings,Melee,1.5sec,Slow
4,P.E.K.K.A,Na,Epic,Ground,Melee,1.8sec,Slow
5,Minions,x 3,Common,Air & Ground,Melee,1.0sec,Fast


In [ ]:
# Creamos un csv
csv_cartas = df.to_csv('cartas1.csv', index=False, encoding='utf-8-sig')